# Location Based Business Reccomendation Engine

In [4]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from tqdm.notebook import tqdm
import pickle
import folium
import sklearn

## Neighborhood Analysis


In [6]:
with open('api.txt') as f:
    CLIENT_ID = f.readline().strip()
    CLIENT_SECRET = f.readline().strip()    

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


FileNotFoundError: [Errno 2] No such file or directory: 'api.txt'

In [ ]:
neighborhood_latitude = present_stores.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = present_stores.loc[0, 'longitude'] # neighborhood longitude value
neighborhood_name = present_stores.loc[0, 'address'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

## Import and Clean Metro Data from Census


In [ ]:
metro_df = pd.read_csv('DP03_All_Metros.csv')
metro_df.head()

In [ ]:
metro_df.dtypes
metro_df[metro_df['geographic area name'] == 'Fort Payne']
#The API returns all Null Values as -9999999999.0, convert them to label average
metro_df.replace(-999999999.0, np.nan, inplace = True)


In [ ]:
for column in metro_df.columns[4:]:
    metro_df[column].replace(np.nan, metro_df[column].mean(), inplace = True)

##  Add Geospatial Data To Metro Data

In [ ]:
metro_df[['geographic area name', 'state']].head()

In [ ]:
geolocator = ArcGIS(user_agent = 'IBM_DS_Capstone')

In [7]:
def locateMetroCities(cities, states):
    '''
    cities: str, name of metro. Eg: "Seattle-Tacoma-Bellevue", or "Idaho Falls"
    states: str, state codes. Eg: "ID-WA", or "OR"
    '''
    
    city_list = cities.split('-')
    state_list = states.split('-')
    
    lat_long_list = []
    
    for city in city_list:
        
        location = geolocator.geocode(city + ', ' + state_list[0])
        
        #makes sure we have the right city in the right state
        if (location == None) or (location.address.split(', ')[0] != city):
            
            i = 1
            while (i < len(state_list)) and (location == None):
                
                location = geolocator.geocode(city + ', ' + state_list[i])
                i += 1
        
        lat_long_list.append([location.latitude, location.longitude])
    
    return lat_long_list


In [ ]:
lat_long = metro_df.loc[0:74].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long2 = metro_df.loc[75:148].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long3 = metro_df.loc[149:222].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long4 = metro_df.loc[223:296].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long5 = metro_df.loc[297:370].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long6 = metro_df.loc[371:444].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long7 = metro_df.loc[445:519].apply(lambda x: zlocateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
print(len(lat_long), len(lat_long2), len(lat_long3), len(lat_long4), len(lat_long5), len(lat_long6), len(lat_long7))

In [ ]:
lat_long = pd.concat([lat_long, lat_long2, lat_long3, lat_long4, lat_long5, lat_long6, lat_long7])

In [ ]:
lat_long.to_csv('lat_long.csv', header = False, index = False)

In [ ]:
lat_long = pd.read_csv('lat_long.csv', header = None)
lat_long.head()

In [ ]:
metro_df.insert(5, 'lat_long', lat_long)
metro_df.head()

In [ ]:
def lat_long_str_to_list_of_floats(lat_long_str):
    
    lat_long_str = lat_long_str.replace('[', '').replace(']', '')    
    lat_long_list = lat_long_str.split(',')
    
    lat_long_pairs_count =  int(len(lat_long_list) / 2)    
    lat_long_pairs = [[] for i in range(lat_long_pairs_count)]
    
    count = 0
    for pair in lat_long_pairs:
        
        pair.append(float(lat_long_list[0 + count]))
        pair.append(float(lat_long_list[1 + count]))
        count += 2
        
    return lat_long_pairs

In [ ]:
metro_df['lat_long'] = pd.Series(map(lat_long_str_to_list_of_floats, metro_df['lat_long']))

## Regions of Interest + FourSquare Data

In [ ]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET,
                VERSION)

categories = requests.get(url).json()

food_category_dict = categories['response']['categories'][3]

In [ ]:
def gen_dict_extract(key, var):
    '''
    https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
    Title: Find all occurrences of a key in nested dictionaries and lists
    by user: hexerei software
    28 MAY 2020
    
    Pulls all values with a specific key from a nested dictionary
    'var' is the dict
    
    '''
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result

In [ ]:
id_list = list(gen_dict_extract('id', food_category_dict))
name_list = list(gen_dict_extract('name', food_category_dict))

#remove generic 'Food' category
id_list = id_list[1:]
name_list = name_list[1:]

In [ ]:
name_id_dict = {}
id_name_dict = {}
for i in range(0,len(name_list)):
    name_id_dict[name_list[i]] = id_list[i]
    id_name_dict[id_list[i]] = name_list[i]

In [ ]:
def getRestaurantCoordsAndType(lat_long_list, category_list, limit = 50, radius = 25000, redo50 = False):

    allVenues_df = pd.DataFrame(columns = ['name', 'category', 'category_id', 'id', 'lat', 'long'])
    
    for lat_long in lat_long_list:
        
        for category in category_list:

            lat = lat_long[0]
            long = lat_long[1]

            url = 'https://api.foursquare.com/v2/venues/explore?&openNow=0&time=any&sortByPopularity=1&categoryId=' + category + '&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                        CLIENT_ID, 
                        CLIENT_SECRET, 
                        VERSION, 
                        lat, 
                        long, 
                        radius, 
                        limit)
            
            result = requests.get(url).json()
            
            #Keep relevant data only
            try:
                #Foursquare will only return a max of 50 venues.
                #If the max is reached, this section splits the initial radial search into several smaller searches
                if 'groups' in result['response'] and len(result['response']['groups'][0]['items']) == 50 and redo50 == False:
                    
                    lat_long_expanded_list = []
                    degree_list = [0, 45, 90, 135, 180, 225, 270, 315]                    
                    for degree in degree_list:
                        if degree % 90 == 0:
                            distance = radius/2
                        else:
                            distance = radius * (2/5)
                        origin = Point(lat, long)
                        lat_long_expanded_list.append(list(geodesic(meters = distance).destination(origin, degree))[0:2])
                    
                    allVenues_df = pd.concat([allVenues_df, getRestaurantCoordsAndType(lat_long_expanded_list, [category], limit = 50, radius = radius/2, redo50 = True)], sort = False)
                    
                elif 'groups' in result['response'] and len(result['response']['groups'][0]['items']) > 0:
                    
                    venues_df = pd.DataFrame(list(gen_dict_extract('venue', result['response']['groups'][0])))
                    
                    venues_df.drop(venues_df.loc[venues_df['categories'].map(lambda x: len(x)== 0)].index, inplace = True)                    
                    venues_df['category'] = venues_df['categories'].map(lambda x: x[0]['name'])
                    venues_df['category_id'] = venues_df['categories'].map(lambda x:x[0]['id'])
                    
                    venues_df['lat'] = venues_df['location'].map(lambda x: x['lat'])
                    venues_df['long'] = venues_df['location'].map(lambda x: x['lng'])
                    
                    venues_df.drop(['categories','location', 'photos'], axis = 1, inplace = True)

                    if 'venuePage' in venues_df:
                        venues_df.drop(['venuePage'], axis = 1, inplace = True)
                    if 'delivery' in venues_df:
                        venues_df.drop(['delivery'], axis = 1, inplace = True)
                    if 'events' in venues_df:
                        venues_df.drop(['events'], axis = 1, inplace = True)
                    
                    allVenues_df = pd.concat([allVenues_df, venues_df], sort = False)
                    
            except KeyError as err:

                print('KeyError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
                
                continue
                
            except IndexError as err:
                
                print('IndexError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
            
            except TypeError as err:
                
                print('TypeError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
    
    allVenues_df.drop_duplicates(subset = 'id', keep = 'last', inplace = True)
    
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Gas Station']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Grocery Store']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Food Court']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Food']
    #allVenues_df.drop(allVenues_df.loc[allVenues_df['category'] == 'Fast Food Restaurant'].index, inplace = True)
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Fast Food Restaurant']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Coffee Shop']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Shopping Plaza']
    
    if redo50 == False:
        allVenues_df.rename(columns = {'id' : 'venue_id'}, inplace = True)
    allVenues_df.reset_index(drop = True, inplace = True)
    
    return allVenues_df

### foursquare API call usage batching 5000 calls per hour

In [ ]:
metro_venue_dict = {}
metro_venue_dict2 = {}
metro_venue_dict3 = {}
metro_venue_dict4 = {}
metro_venue_dict5 = {}

print('Starting batch 1 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[0 : 4].index:    
    metro_venue_dict[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 2 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[5 : 9].index:    
    metro_venue_dict2[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 3 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[10 : 14].index:    
    metro_venue_dict3[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 4 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[15 : 19].index:    
    metro_venue_dict4[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 5 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[20 : 24].index:    
    metro_venue_dict5[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 6 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[25 : 29].index:    
    metro_venue_dict6[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 7 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[30 : 34].index:    
    metro_venue_dict7[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

    
print('Final batch complete at ' + datetime.now().strftime("%H:%M:%S"))

In [ ]:
metro_venue_dict = {**metro_venue_dict, **metro_venue_dict2, **metro_venue_dict3, **metro_venue_dict4, **metro_venue_dict5, **metro_venue_dict6, **metro_venue_dict7}

In [ ]:
#mkdir('./metro_venues')
    
for key in metro_venue_dict.keys():
    metro_venue_dict[key].to_csv('./metro_venues/' + key + '.csv', index = False)


In [ ]:
metro_venue_dict = {}

for file in listdir('./metro_venues'):
    metro_venue_dict[file[0:-4]] = pd.read_csv('./metro_venues/' + file)

metro_venue_dict['Aberdeen'].head()

## K Means Clustering For Restaurant Type Frequency Per Metro

In [ ]:
rest_frequency_df = pd.DataFrame(name_list)
rest_frequency_df.set_index(0, inplace = True)
del rest_frequency_df.index.name

for key in metro_venue_dict.keys():
    
    rest_frequency_df[key] = metro_venue_dict[key]['category'].value_counts()

rest_frequency_df.replace(np.nan, 0, inplace = True)

rest_frequency_df.head()

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
frequency_normed_df = pd.DataFrame(min_max_scaler.fit_transform(rest_frequency_df))

frequency_normed_df.head()


In [ ]:
k = 3

kmeans = KMeans(n_clusters = k).fit(frequency_normed_df.transpose())
kmeans.labels_[0:5]


In [ ]:
try:
    PNW_metro_df.insert(0, 'cluster', kmeans.labels_)
except ValueError:
    PNW_metro_df['cluster'] = kmeans.labels_
    
PNW_metro_df.head()

In [ ]:
PNW_metro_df['cluster'].value_counts()

In [ ]:
map_clusters = folium.Map(width = '100%', height = '100%', location = [45.3, -118], zoom_start = 5)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for i in PNW_metro_df.index:
    lat = PNW_metro_df['lat_long'][i][0][0]
    long = PNW_metro_df['lat_long'][i][0][1]
    metro = PNW_metro_df['geographic area name'][i]
    pop = PNW_metro_df['population_16_up'][i]
    cluster = PNW_metro_df['cluster'][i]
    
    label = folium.Popup(str(metro) + '\nPopulation: ' + str(pop) + '\nCluster: ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Statistics for Each Cluster

In [ ]:
cluster_mean_std = [[[],[]], [[],[]], [[],[]]]

cluster_mean_std[0][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].mean(axis = 1)
cluster_mean_std[0][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].std(axis = 1)

cluster_mean_std[1][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].mean(axis = 1)
cluster_mean_std[1][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].std(axis = 1)

cluster_mean_std[2][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].mean(axis = 1)
cluster_mean_std[2][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].std(axis = 1)


In [ ]:
cluster0_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].mean(axis = 1)
cluster1_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].mean(axis = 1)
cluster2_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].mean(axis = 1)

cluster0_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].std(axis = 1)
cluster1_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].std(axis = 1)
cluster2_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].std(axis = 1)



In [ ]:
cluster0_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']]
cluster1_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']]
cluster2_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']]

cluster0_df.insert(0,'cluster_std', cluster0_std)
cluster0_df.insert(0,'cluster_mean', cluster0_mean)
cluster1_df.insert(0,'cluster_std', cluster1_std)
cluster1_df.insert(0,'cluster_mean', cluster1_mean)
cluster2_df.insert(0,'cluster_std', cluster2_std)
cluster2_df.insert(0,'cluster_mean', cluster2_mean)

In [ ]:
cluster0_df.head()

In [ ]:
#index cluster dataframes by the restaruant type
cluster0_df.index = venue_frequency_df.index
cluster1_df.index = venue_frequency_df.index
cluster2_df.index = venue_frequency_df.index

cluster0_df.head()

## How many standard Deviations is each restaurant type frequency from the mean in each region?

In [ ]:
#subtract mean from frequecy
std0_df = cluster0_df[cluster0_df.columns[2:23]].subtract(cluster0_df['cluster_mean'], axis = 0)
std1_df = cluster1_df[cluster1_df.columns[2:23]].subtract(cluster1_df['cluster_mean'], axis = 0)
std2_df = cluster2_df[cluster2_df.columns[2:23]].subtract(cluster2_df['cluster_mean'], axis = 0)

#divide by std
std0_df = std0_df[std0_df.columns[:]].div(cluster0_df['cluster_std'], axis = 0)
std1_df = std1_df[std1_df.columns[:]].div(cluster1_df['cluster_std'], axis = 0)
std2_df = std2_df[std2_df.columns[:]].div(cluster2_df['cluster_std'], axis = 0)

std_list[0] = std0_df
std_list[1] = std1_df
std_list[2] = std2_df



In [ ]:
std0_df.head()

In [ ]:
std_list[0].head()

## Make reccomendations on metropolitian regions for each restaurant
 by population

In [ ]:
pop_venue_ratio_list = []

for metro in PNW_metro_df['geographic area name']:
    
    pop_venue_ratio_list.append(float(PNW_metro_df[PNW_metro_df['geographic area name'] == metro]['population_16_up']) / len(metro_venue_dict[metro]))


In [ ]:
PNW_metro_df.insert(5, 'pop_venue_ratio', pop_venue_ratio_list)

PNW_metro_df.head()

In [ ]:
top_pop_venue_ratio_df = PNW_metro_df.nlargest(3, ["pop_venue_ratio"], keep = 'all')[['cluster', 'geographic area name', 'pop_venue_ratio']]
top_pop_venue_ratio_df.reset_index(inplace = True, drop = True)
top_pop_venue_ratio_df


In [ ]:
print(std_list[top_pop_venue_ratio_df.iloc[0]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[0]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[0]['geographic area name']], '\n')
print(std_list[top_pop_venue_ratio_df.iloc[1]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[1]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[1]['geographic area name']], '\n')
print(std_list[top_pop_venue_ratio_df.iloc[2]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[2]['geographic

### get best locations for top under represented locations

In [ ]:
top_underrepresented = [[], [], []]

In [ ]:
top_underrepresented[0] = std_list[top_pop_venue_ratio_df.iloc[0]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[0]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[0]['geographic area name']].index.values
top_underrepresented[1] = std_list[top_pop_venue_ratio_df.iloc[1]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[1]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[1]['geographic area name']].index.values
top_underrepresented[2] = std_list[top_pop_venue_ratio_df.iloc[2]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[2]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[2]['geographic area name']].index.values


In [ ]:
metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'] = 'not yet'
metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']]['intrametro cluster'] = 'not yet'
metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']]['intrametro cluster'] = 'not yet'

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']].head()

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'].value_counts()['not yet']


In [8]:
resto_lat_long_list = []
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']][['lat', 'long']].to_numpy())
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']][['lat', 'long']].to_numpy())
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']][['lat', 'long']].to_numpy())
eps = 0.004

dbsacn_list = []
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[0]))
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[1]))
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[2]))

dbsacn_list[0].labels_[0:4]

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'] = dbsacn_list[0].labels_
metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']]['intrametro cluster'] = dbsacn_list[1].labels_
metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']]['intrametro cluster'] = dbsacn_list[2].labels_

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']].head()


NameError: name 'metro_venue_dict' is not defined

### top 10 most populated clusters

In [ ]:
def getMostPopulatedClusters(metro, topXclusters = 10, viewOutliers = False):
    
    cluster_value_counts = pd.DataFrame(metro_venue_dict[metro]['intrametro cluster'].value_counts())
    clusters = list(cluster_value_counts[0:topXclusters].index)

    #Remove -1 (outliers) from clusters
    if -1 in clusters and viewOutliers == False:   
        outlier_index = clusters.index(-1)
        
        if len(cluster_value_counts) > topXclusters:
            clusters[outlier_index] = cluster_value_counts.index[topXclusters]
        else:
            clusters.pop(outlier_index)
        
    return clusters